__Load packages for plotting and data handling__

In [ ]:
using Plots,LaTeXStrings, StatPlots
using DataFrames
using DataFramesMeta
using Interact

__add worker processes for parallel computation__

In [ ]:
addprocs(3)

### Define simulation routines

__Methods for Metropolis__

In [1]:
@everywhere begin
# returns a LxL Matrix filled randomly with ±1
function randomConfiguration(L::Int)
    return rand(Int8[-1,1], L,L)
end

# returns a configuration where all bonds are frustrated
function frustratedConfiguration(L::Int)
    return Int8[(-1)^(i+j) for i=1:L,j=1:L]
end

## Energy of configuration
## Arguments:
##  state    state::Matrix{Int8}
##  coupling J::Float64 defaults to 1.0
##  field    h::Float64 defaults to 0.0
function H(state::Matrix{Int8},J=1.,h=0.)
    L = size(state,1)
    s = 0.
    s = h*(sum(state))
    
    @inbounds for i in 1:L, j in 1:L
        s += -J*state[i,j]*(state[i%L+1,j]+state[i,j%L+1])
    end
    return s
end

## Energy difference between the current state `state`
## and one spin flipped at position `pos::Int`
## Arguments:
##  state    state::Matrix{Int8}
##  pos      linear index of `state` 1..L^2
##  coupling J::Float64 defaults to 1.0
##  field    h::Float64 defaults to 0.0
function dH(state::Matrix{Int8},pos,J=1.,h=0.)
    L = size(state,1)
    i,j = ind2sub(state,pos) #convert the linear index to index pair (i,j)
    @inbounds return 2*J*state[pos]*( state[i%L+1,j]+state[i,j%L+1]+state[i==1?L:i-1,j]+state[i,j==1?L:j-1] ) + 2*h*state[pos]
end

## Magnetization per spin
function m(state) 
    Float64(sum(state)/length(state))
end
    
## Perform one step of the Metropolis algorithm.
## `state` is mutated.
## Arguments:
##  state::Matrix{Int8} Current configuration 
##  beta::Float64       Inverse temperature
##  h::Float64          External field
function metropolis_step!(state::Matrix{Int8},beta,h)
    i = rand(1:length(state))
    dh = dH(state,i,1.,h)
    if dh <= 0 || rand()<exp(-beta*dh)
        state[i] *= -1
    end
    return nothing
end

function sweep!(state,n,beta,h)
    for _ in 1:n
        metropolis_step!(state,beta,h)
    end
end

function init(L,beta,h,sweep)
    state = randomConfiguration(L)
    # Initial sweep to get into the steady state
    sweep!(state,sweep,beta,h)
    return state
end    

## Runs the Metropolis algorithm for a given set of parameters.
## Samples in defined intervals along the Markov-Chain.
## An initial thermal sweep to go to equilibrium may be specified.
## Call like 
##  run_metropolis(50,0.,0.;Tmax=25*10^3*50^2,poll_interval=10*50^2,sweep=10^3*50^2)
##
## Arguments
##  L::Int             Linear system size
##  beta::Float64      Inverse temperature
##  h::Float64         External field
##  Tmax::Int          Number of steps
##  sweep::Int         Length of the initial sweep
##  poll_interval::Int Record the observables every `poll_interval` steps.
##  repeat::Int Restart the chain
    
function run_metropolis(L::Int, beta,h;Tmax::Int=1,sweep::Int=0,poll_interval::Int=1)
    ## Initialise a random state
    state = init(L,beta,h,sweep)
    return _run_metropolis!(state,beta,h,Tmax=Tmax,poll_interval=poll_interval)
end

    
function _run_metropolis!(state::Matrix{Int8},beta,h;Tmax::Int=1,poll_interval::Int=1)
    
    ## Define a matrix in which to record the observables.
    ## One row for each observable, e.g E and m.
    ## Preallocating the matrix gives much better performance than
    ## constructing it on the fly.
    observables = zeros(Float64,5)
    
    k = 0 #counts the number of samples

    # pick the middle lattice site for the spin correlation
        
    t=0 #number of simulation steps
    e=0.
    mag=0.

    @inbounds begin
        while(t<Tmax)
            ## Do the defined no. of steps before
            ## taking a measurement
            sweep!(state,poll_interval,beta,h)
            
            ## Record observables
            e = H(state,1.,h)
            mag = m(state)
            observables[1] += e
            observables[2] += e^2
            observables[3] += mag
            observables[4] += mag^2
            observables[5] += mag^4

            ## increment counters
            k+=1
            t+=poll_interval
        end
    end
    ## Return statistics about the observables
    return observables/k
end
    
end

In [ ]:
# Record timeseries of M
function metropolis_timeseries(L::Int, beta, Tmax, poll_interval=L^2)
    state = init(L,beta,0.,1000*L^2)
    ts = Vector{Float64}(div(Tmax,poll_interval))
    t=0
    k=1
    mag0 = m(state)
    while(t<Tmax)
        sweep!(state,poll_interval,beta,0.)
        ts[k] = m(state)
        k+=1
        t+=poll_interval
    end
    return ts
end

__Methods for the Wolff algorithm__

In [ ]:
@everywhere begin
    function cluster!(cluster_state, state, i,j, p)
        @inbounds begin L = size(state, 1)
            s = state[i,j]
            cluster_state[i,j] = true
            for neighbor in [(i%L+1,j),(i,j%L+1),(i==1?L:i-1,j),(i,j==1?L:j-1)]
                if state[neighbor...] == s && !cluster_state[neighbor...] && rand()<p
                    cluster_state[neighbor...]=true
                    cluster!(cluster_state, state, neighbor..., p)
                end
            end
        end
    end
    
    function wolff_step!(state, cluster_state, beta)
        @inbounds begin
            L = size(state,1)
            i,j = rand(1:L,2)
            fill!(cluster_state, false)
            cluster!(cluster_state, state, i,j, 1.-exp(-2.*beta))
            state[cluster_state] *= -1
        end
        return nothing
    end
    
    function _run_wolff!(state::Matrix{Int8},cluster,beta,h;Tmax::Int=1,poll_interval::Int=1)

        ## Define a matrix in which to record the observables.
        ## One row for each observable, e.g E and m.
        ## Preallocating the matrix gives much better performance than
        ## constructing it on the fly.
        observables = zeros(Float64,5)

        k = 0 #counts the number of samples

        # pick the middle lattice site for the spin correlation

        t=0 #number of simulation steps
        r=0 #repeat counter
        e=0.
        mag=0.
        @inbounds begin
            while(t<Tmax)
                ## Do the defined no. of steps before
                ## taking a measurement
                for _ in 1:poll_interval
                    wolff_step!(state, cluster, beta)
                end
                ## Record observables
                e = H(state,1.,0.)
                mag = m(state)|>abs
                observables[1] += e
                observables[2] += e^2
                observables[3] += mag
                observables[4] += mag^2
                observables[5] += mag^4

                ## increment counters
                k+=1
                t+=poll_interval
            end
        end
        ## Return statistics about the observables
        return observables/k
    end
    
    function run_wolff(L::Int, beta,h;Tmax::Int=1,sweep::Int=0,poll_interval::Int=1)
        ## Initialise a random state
        function init(L,beta)
            state = randomConfiguration(L)
            cluster = zeros(Bool,L,L)
            # Initial sweep to get into the steady state
            for _ in 1:sweep
                wolff_step!(state, cluster, beta)
            end
            return state,cluster
        end
        
        state,cluster = init(L,beta)
        return _run_wolff!(state,cluster,beta,0.,Tmax=Tmax,poll_interval=poll_interval)
    end    
    
end

---

__Onsager critical temperature__

In [ ]:
Tc = 2/log(1+sqrt(2));

### Autocorrelation

In [ ]:
let Tmax=10000,L=100
@time  ts = mapreduce(vcat, [4.,2.5,2.26,2.0]) do T
    df = DataFrame()
    df[:t] = collect(1:Tmax-1)
    df[:L] = L
    df[:T] = T

    ts = metropolis_timeseries(L,1/T,Tmax*L^2)
    ts -=  mean(ts)

    df[:autocor] = map(1:Tmax-1) do t
        1./(Tmax-t+1)*sum(ts[1:Tmax-t+1].*ts[t:Tmax])
    end
    df[:autocor] ./= 1.#df[1,:autocor]
    df
end
end;

In [ ]:
@df @where(ts,:autocor.>0.) plot(:t, :autocor, group=:T, xscale=:log10, xlim=(1,3000), size=(1024,768));

### Histogram

In [ ]:
magnetization = DataFrame()
let Tmax=50000,L=100
@time  magnetization = mapreduce(vcat, [4.,2.5,Tc,2.0,1.0]) do T
    df = DataFrame()
    df[:t] = collect(1:Tmax)
    df[:L] = L
    df[:T] = T

    df[:m] = metropolis_timeseries(L,1/T,Tmax*L^2)
    #ts -=  mean(ts)

    df
end
end;

In [ ]:
magnetization = readtable("magnetization-ts-N100.tsv",separator='\t')

In [ ]:
writetable("magnetization-ts-N100.tsv",magnetization,separator='\t')

In [ ]:
@df @where(magnetization, :T.==2.0) plot(:t,(:m))

In [ ]:
@df magnetization histogram(:m, group=:T,alpha=0.6, normalize=false, size=(1024,768));

### Simulation

In [ ]:
obs = DataFrame(Float64,0,7)
names!(obs,[:L,:T,:e,:e2,:m,:m2,:m4])

## Temperature range
Ts = linspace(2.6,2.0,50)
Ls = [10,30,50,100]

for L in Ls
@everywhere begin state=randomConfiguration($L); end
@time map(pmap(Ts) do T
        println("T=$T")
            
#       result = run_wolff(L, 1/T,0.;Tmax=10*10^3,sweep=100*L^2,poll_interval=1)
#       result = run_metropolis(L, 1/T,0.;Tmax=50*10^3*L^2,sweep=1000*L^2,poll_interval=10*L^2)
            
        sweep!(state,1000*(L^2),1/T,0.)            
        result = _run_metropolis!(state, 1/T,0.;Tmax=1000*10^3*L^2,poll_interval=100*L^2)

        E = result[1]
        E2 = result[2]
        mag = result[3]
        mag2 = result[4]
        mag4 = result[5]
        [L,T,E,E2,mag,mag2,mag4]'
    end) do row
        #println(row)
        push!(obs,row);
    end;
end
   
# obs[:t] = obs[:T]-Tc;


__Calculate susc. & heat capacity__

In [ ]:
obs[:c] = (obs[:e2].-obs[:e].^2)./(obs[:T].^2)./obs[:L].^2;
obs[:chi] = (obs[:m2].-obs[:m].^2)./obs[:T].*obs[:L].^2;

__Save/load data__

In [ ]:
writetable("metropolis04-N10-30-50-100-longWait.tsv", obs)

In [ ]:
obs = readtable("metropolis04-N10-30-50-100-longWait.tsv");

In [ ]:
tail(obs)

### Analysis

__Estimate the critical temperature from the heat capacity__

In [ ]:
@df obs plot(:T, abs.(:c), group=:L, legend=true,line=(:dash, 2.), marker=(:auto,3), ylabel="spec. heat capacity")
vline!([Tc Tc+0.006],lab=["Onsager" "numerical"], lw=3)

In [ ]:
# Estimated critical temperature
Tcnum = Tc + 0.006

__Define reduced temperature__

In [ ]:
obs[:t] = obs[:T]-Tcnum;

__Plot observables__

In [ ]:
@manipulate for L in levels(obs[:L])
@df obs[obs[:L].==L,:] plot(:T, [abs.(:m) :chi :e :c],
    group=:L, layout=4, legend=false, size=(1024,768),
    marker=:square,ms=2., line=(:dash, 2.), ylabel=[L"\vert M \vert" L"\chi" L"E/N" L"c_h"])
xlabel!(L"T-T_c")
vline!([[Tcnum] [Tcnum] [Tcnum]],lab="Onsager")
end

__Split observables in sets above/below $T_c$ __

In [ ]:
obs_above = obs[(obs[:t].>0) .& (obs[:L].==100),:];
obs_below = obs[(obs[:t].<0) .& (obs[:L].==100),:];

### Critical exponents

In [ ]:
# General Linear Models is used to perform linear regressions.
# A simpler package is LsqFit, but it can't deal with DataFrames directly.
using GLM

__Magnetization__

In [ ]:
log_obs = @select(obs_below,logt=log.(abs.(:t)),logm=log.(abs.(:m)));

In [ ]:
beta_regr = lm(@formula(logm ~ logt),log_obs[3:end,:]) # neglect the two datapoints closest to Tc

In [ ]:
scatter(log.(abs.(obs_below[:t])),log.(abs.(obs_below[:m])), xlabel="log|T-Tc|", ylabel="log M/N",legend=false,size=(800,600))
#scatter!(log.(abs.(obs_above[:t])),log.(abs.(obs_above[:m])), xlabel="log|T-Tc|", ylabel="log\\chi",legend=false)

plot!(-7.:-1,t->t*1/8+0.08, lw=3) # exact exponent 1/8
#plot!(-5.:-1,t->abs(t)*7/4-11)

__Suscpetibility__

In [ ]:
scatter(log.(abs.(obs_below[:t])),log.(abs.(obs_below[:chi])), xlabel="log|T-Tc|", legend=true, marker=:auto, lab="below",size=(800,600))
scatter!(log.(abs.(obs_above[:t])),log.(abs.(obs_above[:chi])), xlabel="log|T-Tc|", ylabel="log\\chi", marker=(2,:auto), lab="above")

plot!(-5.:-1,t->-t*7/4+0.6, lab="above", lw=3)
plot!(-5.:-1,t->-t*7/4-3., lab="below", lw=3)

__Heat capacity__

In [ ]:
scatter(log.(abs.(obs_below[:t])),log.(obs_below[:c]), xlabel="log|T-Tc|", legend=false, size=(800,600))
scatter!(log.(abs.(obs_above[:t])),log.(obs_above[:c]), xlabel="log|T-Tc|", ylabel="log C",legend=false)
#
plot!(linspace(-5,-1,50),t->-1/2*t-0.9,lw=3.)
plot!(linspace(-5,-1,50),t->1.*log(-t)-0.6,lw=3.)


### Addendum

__Binder cumulants__

In the unordered phase the distribution of $m$ is a Gaussian around $0$ of width $~1/\sqrt{t}$, while deep in the ordered phase it is _two_ Gaussians peaked at $\pm 1$.



In [ ]:
@df obs plot(:T, 1.-:m4./3./:m2.^2, group=:L, marker=(:auto, 0.),size=(800,600))
vline!([Tc-0.005])

## Real time visualization

In [2]:
using GLVisualize, GeometryTypes

In [3]:
using IterTools,Colors,Reactive,Interact

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b92f919d597277debc5ed4c6eb2190cd1960c521-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b92f919d597277debc5ed4c6eb2190cd1960c521-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [53]:
# This function maps the spin state to an array of colors
function color_gen(v0,basecolor)
    map(v0) do x
        if x==1
            RGB(0f0,0f0,0f0)
        elseif x==-1.
            RGB(1f0,min(1f0,Float32(basecolor)),0f0)
        end
    end
end

color_gen (generic function with 1 method)

In [5]:
# Reset window
function reset_window()
try
    empty!(window)
    close(color_signal)
    close(state_map)
    close(timesignal)
    close(temperature)
catch UndefVarError
end
end

reset_window (generic function with 1 method)

In [6]:
function adjust_cam!(window;eyepos_vec=Vec3f0(0,0,+1),lookat_vec=Vec3f0(0,0,0),up_vec=cross(lookat_vec-eyepos_vec,-Vec3f0(1,0,0)))
    push!(window.cameras[:perspective].eyeposition, eyepos_vec)
    push!(window.cameras[:perspective].lookat, lookat_vec)
    push!(window.cameras[:perspective].up, up_vec)
    push!(window.cameras[:perspective].fov, 90)
end

adjust_cam! (generic function with 1 method)

__Temperature & field__

In [75]:
a_signal

519: "input-290" = 0.5 Float64 

In [70]:
# Gives us some sliders to manipulate parameters
temperature_slider = slider(0.:0.01:10., label="temperature", value=2.26)
temperature = observe(temperature_slider)
temperature_signal = Signal(2.26)
on(val->push!(temperature_signal,val), temperature)
display(temperature_slider)

h_slider = slider(-5.:0.1:+5., label="field")
h = observe(h_slider)
h_signal = Signal(0.)
on(val->push!(h_signal,val), h)
display(h_slider)

a_slider = slider(0.:0.1:2., label="simulation speed", value=0.)
a = observe(a_slider)
a_signal = Signal(0.)
on(val->push!(a_signal,val), a)
display(a_slider)

(div { className="field" }
    (div { attributes=Dict("style"=>"display:flex; justify-content:center; align-items:center;") }
    (div { attributes=Dict("style"=>"text-align:right;width:18%") }
      (label { className="interact " style=Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")) }
        "temperature"))
    (div { attributes=Dict("style"=>"flex-grow:1; margin: 0 2%") }
      (input { max=10.0 min=0.0 attributes=Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}")) step=0.01 className="slider slider is-fullwidth" style=Dict{Any,Any}() }))
    (div { attributes=Dict("style"=>"width:18%") }
      (p { attributes=Dict("data-bind"=>"text: displayedvalue") }))))

(div { className="field" }
    (div { attributes=Dict("style"=>"display:flex; justify-content:center; align-items:center;") }
    (div { attributes=Dict("style"=>"text-align:right;width:18%") }
      (label { className="interact " style=Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")) }
        "field"))
    (div { attributes=Dict("style"=>"flex-grow:1; margin: 0 2%") }
      (input { max=5.0 min=-5.0 attributes=Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}")) step=0.1 className="slider slider is-fullwidth" style=Dict{Any,Any}() }))
    (div { attributes=Dict("style"=>"width:18%") }
      (p { attributes=Dict("data-bind"=>"text: displayedvalue") }))))

(div { className="field" }
    (div { attributes=Dict("style"=>"display:flex; justify-content:center; align-items:center;") }
    (div { attributes=Dict("style"=>"text-align:right;width:18%") }
      (label { className="interact " style=Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")) }
        "simulation speed"))
    (div { attributes=Dict("style"=>"flex-grow:1; margin: 0 2%") }
      (input { max=2.0 min=0.0 attributes=Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}")) step=0.1 className="slider slider is-fullwidth" style=Dict{Any,Any}() }))
    (div { attributes=Dict("style"=>"width:18%") }
      (p { attributes=Dict("data-bind"=>"text: displayedvalue") }))))

In [134]:
# 3.85 is the transition field strength at T=0.03
h[] = +3.85

3.85

__Prepare the initial state__

*Make sure to call `reset_window()` before changing the system size!*

In [87]:
reset_window()

In [88]:
L=128
config0 = frustratedConfiguration(L);
cluster = zeros(Bool, L,L)
#config0 = randomConfiguration(L);
sweep!(config0,0,1/temperature[],0.)

__Prepare the window, signals and primitives__

In [95]:
if !isdefined(:window) || !isopen(window)
    window=glscreen(resolution=(800,800))
end

reset_window()

target_fps = 60
timesignal = fps(target_fps)

state_map = Reactive.map((_,T,h)->sweep!(config0,div(a[]*L^2,target_fps),1/T,h), timesignal,temperature_signal,h_signal);
color_signal=map(_->reshape(color_gen(config0,0.3),L^2,1)[:,1], timesignal)

#position = Point2f0[Point2f0(800/L*(xi+1/2),800/L*(yi+1/2)) for (xi,yi) in product(0:L-1,0:L-1)]
position = Point3f0[Point3f0(2*xi/L-1.,2*yi/L-1.,0) for (xi,yi) in product(0:L-1,0:L-1)]

#square = HyperRectangle(Vec2f0(0),Vec2f0(800/L));
square = HyperRectangle(Vec3f0(0.),Vec3f0(2/L,2/L,0));
circle = HyperSphere(Point3f0(0),2f0/L);

__Visualize the lattice__

In [93]:
# [diffuse_color, specular_color, ambient_color, position]
light  = Vec3f0[Vec3f0(0.0,0.0,0.0), Vec3f0(0.1,0.1,0.1), Vec3f0(1.0,1.0,1.0), Vec3f0(0,0,100)];

In [30]:
light=Signal(light)

208: "input-127" = GeometryTypes.Vec{3,Float32}[Float32[1.0, 1.0, 1.0], Float32[0.1, 0.1, 0.1], Float32[1.0, 1.0, 1.0], Float32[0.0, 0.0, 10.0]] Array{GeometryTypes.Vec{3,Float32},1} 

In [13]:
using GLAbstraction

In [96]:
lattice = visualize((square,position),color=color_signal);
lattice.children[1].uniforms[:light] = light;

In [97]:
_view(lattice,window,camera=:perspective)

4-element Array{GeometryTypes.Vec{3,Float32},1}:
 Float32[0.0, 0.0, 0.0]  
 Float32[0.1, 0.1, 0.1]  
 Float32[1.0, 1.0, 1.0]  
 Float32[0.0, 0.0, 100.0]

__Adjust the camera to look from above into the xy-plane__

In [65]:
adjust_cam!(window,eyepos_vec=Vec3f0(0,0,-1))

__Start the render loop__

In [36]:
@async renderloop(window)

Task (runnable) @0x000000011cac7850